### Import packages

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import csv
import numpy as np
import os
import requests
from io import StringIO
import pymc as pm
import arviz as az
from tqdm import tqdm
import hssm
import random
from hssm.distribution_utils import make_distribution
from hssm.likelihoods import DDM
from scipy.stats import pearsonr
import pytensor.tensor as pt
from pytensor.scan import scan
from scipy.stats import norm, halfnorm, gamma
from pathlib import Path

### Set variables

In [2]:
n_rounds = 160
n_indi_param_sets = 21 
n_group_param_sets = 30

fixed_ndt = 0.25
wald_prior_class = "log_normal_prior_centered"


### Import fitted parameters and gambles

In [3]:
# Experiment version
exp_version = 2

#Nextcloud credentials
username = 'algarrid'
password = 'faunistico'

# URL of the CSV file 
file_url = 'https://cloud.ilabt.imec.be/remote.php/dav/files/af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/risky_dm_2/Gambles/gambles_risky_dm_2.csv'

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_gambles = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Gambles downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

Gambles downloaded


## Import group fitted parameters

In [4]:
df_wald_group_fitted_params = pd.read_csv(f'../model_fitting/fitted_parameters/wald/group_fitted_params_wald_{wald_prior_class}.csv')

## Generate parameter sets 

##### Generate group-level parameter sets

In [5]:
rows = []

for g in range(n_group_param_sets):

    if wald_prior_class in ["log_normal_prior_centered", "log_normal_prior_non_centered"]:

        group_log_beta0_mu_mean = df_wald_group_fitted_params['group_log_beta0_mu_mean'].iloc[0]
        group_log_beta0_mu_sd = df_wald_group_fitted_params['group_log_beta0_mu_sd'].iloc[0]
        group_log_beta0_mu = norm(loc=group_log_beta0_mu_mean, scale=group_log_beta0_mu_sd).rvs()

        group_log_beta1_mu_mean = df_wald_group_fitted_params['group_log_beta1_mu_mean'].iloc[0]
        group_log_beta1_mu_sd = df_wald_group_fitted_params['group_log_beta1_mu_sd'].iloc[0]
        group_log_beta1_mu = norm(loc=group_log_beta1_mu_mean, scale=group_log_beta1_mu_sd).rvs()

        group_log_beta2_mu_mean = df_wald_group_fitted_params['group_log_beta2_mu_mean'].iloc[0]
        group_log_beta2_mu_sd = df_wald_group_fitted_params['group_log_beta2_mu_sd'].iloc[0]
        group_log_beta2_mu = norm(loc=group_log_beta2_mu_mean, scale=group_log_beta2_mu_sd).rvs()
        
        group_log_drift_mu_mean = df_wald_group_fitted_params['group_log_drift_mu_mean'].iloc[0]
        group_log_drift_mu_sd = df_wald_group_fitted_params['group_log_drift_mu_sd'].iloc[0]
        group_log_drift_mu = norm(loc=group_log_drift_mu_mean, scale=group_log_drift_mu_sd).rvs()

        group_log_beta0_sigma_mean = df_wald_group_fitted_params['group_log_beta0_sigma_mean'].iloc[0]
        group_log_beta0_sigma_sd = df_wald_group_fitted_params['group_log_beta0_sigma_sd'].iloc[0]
        group_log_beta0_sigma = norm(loc=group_log_beta0_sigma_mean, scale=group_log_beta0_sigma_sd).rvs()

        group_log_beta1_sigma_mean = df_wald_group_fitted_params['group_log_beta1_sigma_mean'].iloc[0]
        group_log_beta1_sigma_sd = df_wald_group_fitted_params['group_log_beta1_sigma_sd'].iloc[0]
        group_log_beta1_sigma = norm(loc=group_log_beta1_sigma_mean, scale=group_log_beta1_sigma_sd).rvs()

        group_log_beta2_sigma_mean = df_wald_group_fitted_params['group_log_beta2_sigma_mean'].iloc[0]
        group_log_beta2_sigma_sd = df_wald_group_fitted_params['group_log_beta2_sigma_sd'].iloc[0]
        group_log_beta2_sigma = norm(loc=group_log_beta2_sigma_mean, scale=group_log_beta2_sigma_sd).rvs()
        
        group_log_drift_sigma_mean = df_wald_group_fitted_params['group_log_drift_sigma_mean'].iloc[0]
        group_log_drift_sigma_sd = df_wald_group_fitted_params['group_log_drift_sigma_sd'].iloc[0]
        group_log_drift_sigma = norm(loc=group_log_drift_sigma_mean, scale=group_log_drift_sigma_sd).rvs()
    
        rows.append({
            'group_parameter_set_ID': g,
            'group_log_beta0_mu': group_log_beta0_mu,
            'group_log_beta1_mu': group_log_beta1_mu,
            'group_log_beta2_mu': group_log_beta2_mu,
            'group_log_drift_mu': group_log_drift_mu,
            'group_log_beta0_sigma': group_log_beta0_sigma,
            'group_log_beta1_sigma': group_log_beta1_sigma,
            'group_log_beta2_sigma': group_log_beta2_sigma,
            'group_log_drift_sigma': group_log_drift_sigma,
        })

    else:
        alpha_beta0 = 3.45
        beta_beta0 = 0.5
        group_beta0_mu = gamma(a=alpha_beta0, scale=1/beta_beta0).rvs()
    
        alpha_beta1 = 2.6
        beta_beta1 = 3.2
        group_beta1_mu = gamma(a=alpha_beta1, scale=1/beta_beta1).rvs()
    
        alpha_beta2 = 2.6
        beta_beta2 = 3.2
        group_beta2_mu = gamma(a=alpha_beta2, scale=1/beta_beta2).rvs()
    
        alpha_drift = 4.7
        beta_drift = 3.6
        group_drift_mu = gamma(a=alpha_drift, scale=1/beta_drift).rvs()
        
    
        group_beta0_sigma = halfnorm(scale=0.5).rvs()
        group_beta1_sigma = halfnorm(scale=0.5).rvs()
        group_beta2_sigma = halfnorm(scale=0.5).rvs()
        group_drift_sigma = halfnorm(scale=0.5).rvs()
    
    
    
        # ===== subject-level Gamma draws (positive, no logs) =====
        #beta0 = pm.Gamma("indi_beta0", mu=beta0_mu, sigma=beta0_sigma, dims="subject")
        #beta1 = pm.Gamma("indi_beta1", mu=beta1_mu, sigma=beta1_sigma, dims="subject")
        #beta2 = pm.Gamma("indi_beta2", mu=beta2_mu, sigma=beta2_sigma, dims="subject")
        #drift = pm.Gamma("indi_drift", mu=drift_mu, sigma=drift_sigma, dims="subject")
    
    
        rows.append({
            'group_parameter_set_ID': g,
            'synthetic_group_beta0_mu': group_beta0_mu,
            'synthetic_group_beta1_mu': group_beta1_mu,
            'synthetic_group_beta2_mu': group_beta2_mu,
            'synthetic_group_beta0_sigma': group_beta0_sigma,
            'synthetic_group_beta1_sigma': group_beta1_sigma,
            'synthetic_group_beta2_sigma': group_beta2_sigma,
            'synthetic_group_drift_mu': group_drift_mu,
            'synthetic_group_drift_sigma': group_drift_sigma,
        })


df_synthetic_group_parameters = pd.DataFrame(rows)

out_dir = Path("synthetic_data")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / f"wald_model_synthetic_group_parameters_{wald_prior_class}.csv"
df_synthetic_group_parameters.to_csv(out_path, index=False)

##### Generate individual-level parameter sets

In [6]:
rows = []

for g in range(n_group_param_sets):

    group_level_params_df = df_synthetic_group_parameters[df_synthetic_group_parameters['group_parameter_set_ID']==g]

    if wald_prior_class in ["log_normal_prior_non_centered","log_normal_prior_centered"]:
        
        group_log_beta0_mu = group_level_params_df['group_log_beta0_mu'].iloc[0]
        group_log_beta1_mu = group_level_params_df['group_log_beta1_mu'].iloc[0]
        group_log_beta2_mu = group_level_params_df['group_log_beta2_mu'].iloc[0]
    
        group_log_beta0_sigma = group_level_params_df['group_log_beta0_sigma'].iloc[0]
        group_log_beta1_sigma = group_level_params_df['group_log_beta1_sigma'].iloc[0]
        group_log_beta2_sigma = group_level_params_df['group_log_beta2_sigma'].iloc[0]
    
        group_log_drift_mu = group_level_params_df['group_log_drift_mu'].iloc[0]
        group_log_drift_sigma = group_level_params_df['group_log_drift_sigma'].iloc[0]


        for i in range(n_indi_param_sets):
            
            # Sample
            if wald_prior_class == "log_normal_prior_centered":
                
                indi_log_beta0 = norm(loc=group_log_beta0_mu, scale=group_log_beta0_sigma).rvs()
                indi_log_beta1 = norm(loc=group_log_beta1_mu, scale=group_log_beta1_sigma).rvs()
                indi_log_beta2 = norm(loc=group_log_beta2_mu, scale=group_log_beta2_sigma).rvs()
                indi_log_drift = norm(loc=group_log_drift_mu, scale=group_log_drift_sigma).rvs()
                
            elif wald_prior_class == "log_normal_prior_non_centered":
                z_b0 = norm(loc=0, scale=1).rvs()
                z_b1 = norm(loc=0, scale=1).rvs()
                z_b2 = norm(loc=0, scale=1).rvs()
                z_d  = norm(loc=0, scale=1).rvs()
    
                indi_log_beta0 = group_log_beta0_mu + group_log_beta0_sigma * z_b0
                indi_log_beta1 = group_log_beta1_mu + group_log_beta1_sigma * z_b1
                indi_log_beta2 = group_log_beta2_mu + group_log_beta2_sigma * z_b2
                indi_log_drift = group_log_drift_mu + group_log_drift_sigma * z_d
    
    
            # Transform
            indi_beta0 = np.exp(indi_log_beta0) 
            indi_beta1 = np.exp(indi_log_beta1) 
            indi_beta2 = np.exp(indi_log_beta2) 
            indi_drift = np.exp(indi_log_drift) 
    
            # Synthetic DDM parameters
    
            synthetic_a = np.random.uniform(0.5,3.5)
            synthetic_t = np.random.uniform(0.2,2)
            synthetic_v_diff_p = np.random.uniform(0,4)
            synthetic_v_diff_x = np.random.uniform(0,4)
            synthetic_v_diff_ev = np.random.uniform(0,4)
    
    
            rows.append({
                'group_parameter_set_ID': g,
                'synthetic_par_ID': i,
                'synthetic_indi_beta0': indi_beta0,
                'synthetic_indi_beta1': indi_beta1,
                'synthetic_indi_beta2': indi_beta2,
                'synthetic_indi_drift': indi_drift,
                'synthetic_indi_a': synthetic_a,
                'synthetic_indi_t': synthetic_t,
                'synthetic_indi_v_diff_p': synthetic_v_diff_p,
                'synthetic_indi_v_diff_x': synthetic_v_diff_x,
                'synthetic_indi_v_diff_ev': synthetic_v_diff_ev,
            })
    else:
        group_beta0_mu = group_level_params_df['synthetic_group_beta0_mu'].iloc[0]
        group_beta1_mu = group_level_params_df['synthetic_group_beta1_mu'].iloc[0]
        group_beta2_mu = group_level_params_df['synthetic_group_beta2_mu'].iloc[0]
    
        group_beta0_sigma = group_level_params_df['synthetic_group_beta0_sigma'].iloc[0]
        group_beta1_sigma = group_level_params_df['synthetic_group_beta1_sigma'].iloc[0]
        group_beta2_sigma = group_level_params_df['synthetic_group_beta2_sigma'].iloc[0]
    
        group_drift_mu = group_level_params_df['synthetic_group_drift_mu'].iloc[0]
        group_drift_sigma = group_level_params_df['synthetic_group_drift_sigma'].iloc[0]
    
    
        for i in range(n_indi_param_sets):
            
    
            # sMple
    
            beta0 = gamma(a=(group_beta0_mu/group_beta0_sigma)**2, scale=(group_beta0_sigma**2)/group_beta0_mu).rvs() 
            beta1 = gamma(a=(group_beta1_mu/group_beta1_sigma)**2, scale=(group_beta1_sigma**2)/group_beta1_mu).rvs() 
            beta2 = gamma(a=(group_beta2_mu/group_beta2_sigma)**2, scale=(group_beta2_sigma**2)/group_beta2_mu).rvs() 
            drift = gamma(a=(group_drift_mu/group_drift_sigma)**2, scale=(group_drift_sigma**2)/group_drift_mu).rvs() 
    
            # Synthetic DDM parameters
    
            synthetic_a = np.random.uniform(0.5,3.5)
            synthetic_t = np.random.uniform(0.2,2)
            synthetic_d_p = np.random.uniform(0,4)
            synthetic_d_x = np.random.uniform(0,4)
            synthetic_d_ev = np.random.uniform(0,4)
    
    
            rows.append({
                'group_parameter_set_ID': g,
                'synthetic_par_ID': i,
                'synthetic_indi_beta0': beta0,
                'synthetic_indi_beta1': beta1,
                'synthetic_indi_beta2': beta2,
                'synthetic_indi_drift': drift,
                'synthetic_indi_a': synthetic_a,
                'synthetic_indi_t': synthetic_t,
                'synthetic_indi_v_diff_p': synthetic_v_diff_p,
                'synthetic_indi_v_diff_x': synthetic_v_diff_x,
                'synthetic_indi_v_diff_ev': synthetic_v_diff_ev,
            })

df_synthetic_indi_parameters = pd.DataFrame(rows)

out_dir = Path("synthetic_data")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / f"wald_model_synthetic_indi_parameters_{wald_prior_class}.csv"
df_synthetic_indi_parameters.to_csv(out_path, index=False)

## Simulate data

In [7]:
synthetic_data_list = []

for g in tqdm(range(n_group_param_sets)):

    df_group_indi_parameters = df_synthetic_indi_parameters[df_synthetic_indi_parameters['group_parameter_set_ID']==g]

    for i in range(n_indi_param_sets):
        # Get individual level parameters
        df_par_params = df_group_indi_parameters[df_group_indi_parameters['synthetic_par_ID']==i]

        # Get Wald model parameters
        synthetic_beta0 = df_par_params['synthetic_indi_beta0'].iloc[0]
        synthetic_beta1 = df_par_params['synthetic_indi_beta1'].iloc[0]
        synthetic_beta2 = df_par_params['synthetic_indi_beta2'].iloc[0]
        synthetic_wald_drift = df_par_params['synthetic_indi_drift'].iloc[0]
        
        synthetic_wald_ndt = fixed_ndt 

        # Get DDM parameters
        synthetic_a = df_par_params['synthetic_indi_a'].iloc[0]
        synthetic_ddm_t = df_par_params['synthetic_indi_t'].iloc[0]
        synthetic_v_diff_p = df_par_params['synthetic_indi_v_diff_p'].iloc[0]
        synthetic_v_diff_x = df_par_params['synthetic_indi_v_diff_x'].iloc[0]
        synthetic_v_diff_ev = df_par_params['synthetic_indi_v_diff_ev'].iloc[0]

        # Decide a random order of gamble presentation:
        random_gamble_presentation = np.random.permutation(np.arange(1, 161))


        for n, gam in enumerate(random_gamble_presentation):
         
            gamble_df = df_gambles[df_gambles['GambleNumber']==gam]
    
            diff_x = gamble_df["lot_1_val"].iloc[0] - gamble_df["lot_0_val"].iloc[0]
            
            diff_p = gamble_df["lot_1_prob"].iloc[0] - gamble_df["lot_0_prob"].iloc[0]
    
            diff_x = diff_x/100
            diff_p = diff_p/100
    
            ev_0 = (gamble_df["lot_0_val"].iloc[0]/100) * (gamble_df["lot_0_prob"].iloc[0]/100)
            ev_1 = (gamble_df["lot_1_val"].iloc[0]/100) * (gamble_df["lot_1_prob"].iloc[0]/100)
    
            diff_ev = ev_1 - ev_0
    
            #round_v = synthetic_v_diff_p*diff_p + synthetic_v_diff_x*diff_x 
            round_v = synthetic_v_diff_ev*diff_ev + synthetic_v_diff_p*diff_p + synthetic_v_diff_x*diff_x 
    
            #Simulate ddm
            ddm_sim = hssm.simulate_data(
                model="ddm",
                theta=dict(
                    v = round_v,
                    a = synthetic_a,
                    z = 0.5,
                    t = synthetic_ddm_t,
                ),
                size=1,
                random_state = random.randint(0, 999999)
            )
            
            sim_rt = ddm_sim['rt'].iloc[0]
            sim_response = ddm_sim['response'].iloc[0]

            # Figure out the deadline
            original_deadline = gamble_df['Deadline'].iloc[0]
    
            if sim_rt < original_deadline:
                sim_deadline = original_deadline
            else:
                if sim_rt < 8:
                    sim_deadline = 8
                else:
                    if sim_rt < 10:
                        sim_deadline = 10
                    else:
                        sim_deadline = sim_rt
        
            sim_wt = sim_deadline - sim_rt
    
            # Calculate Wald distribution parameters
            alpha_threshold = synthetic_beta0 + synthetic_beta1*sim_rt + synthetic_beta2*sim_wt

            mu = alpha_threshold/synthetic_wald_drift
            
            lam = alpha_threshold**2
    
            # Simulate reproduced time using PyMC's Wald distribution
            sim_repro_t = pm.draw(pm.Wald.dist(mu=mu, 
                                               lam=lam, 
                                               alpha = synthetic_wald_ndt), 
                                  draws=1).item() 
            
            #Save synthetic data
            synthetic_data_list.append({
                'group_parameter_set_ID': g,
                'synthetic_par_ID': i,
                'Round_ID': n,
                'Gamble_ID': gam,
                'rt': sim_rt,
                'response': sim_response,
                'repro_t': sim_repro_t,
                'wt': sim_wt,
                'diff_x': diff_x,
                'diff_p': diff_p,
                'diff_ev': diff_ev
            })


# Convert the list into a DataFrame
df_synthetic_data = pd.DataFrame(synthetic_data_list)

100%|██████████| 30/30 [25:47<00:00, 51.58s/it]


In [8]:
# make sure the folder exists
outdir = "synthetic_data"
os.makedirs(outdir, exist_ok=True)

# save to CSV
out_path = os.path.join(outdir, f"synthetic_data_{wald_prior_class}.csv")
df_synthetic_data.to_csv(out_path, index=False)